In [1]:
adj_list=[[1,6], [0,2], [1,3], [2, 4], [3, 5], [4,6], [5,0]]

In [2]:
import torch
import random
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
w=3 #window size
d=2 #embedding size
y=2 #walks per vertex
t=6 #walk length
size_vertex=7 

In [4]:
v=[0,1,2,3,4,5,6]


In [5]:
phi=torch.rand((size_vertex, d))

In [6]:
def RandomWalk(v,t):
    walk=[v]
#     print("v= ",v)
#     print("walk= ",walk)
    for i in range(t-1):
        v=adj_list[v][random.randint(0,1)]
        walk.append(v)
#         print("v= ",v)
#         print("walk= ",walk)
    return walk

In [7]:
RandomWalk(2,t)

[2, 3, 4, 5, 4, 3]

In [8]:
# def SkipGram(wvi,  w):
#     for j in range(len(wvi)):
#         for k in range(max(0,j-w) , min(j+w, len(wvi)):
            

In [9]:
# for i in range(y):
#     random.shuffle(v)
#     for vi in v:
#         wvi=RandomWalk(vi,t)
#         SkipGram(wvi, w)

In [ ]:
# skip gram model is defined below:

In [10]:
def pair_create(wvi,  w):
    idx_pairs = []
    for center_node_pos in range(len(wvi)):
        for i in range(-w,w+1):
            context_node_pos=center_node_pos+i
#             print("cont_node_pos ____befo= ",context_node_pos)
            if context_node_pos<0 or context_node_pos>=len(wvi) or context_node_pos==center_node_pos:
                continue
            #context_word_idx = indices[context_word_pos]
#             print("cont_node_pos= ",context_node_pos)
            idx_pairs.append((wvi[center_node_pos], wvi[context_node_pos]))
#         print("===================================")
#         print(idx_pairs)

#     idx_pairs = np.array(idx_pairs)
    return idx_pairs
            

In [11]:
len(pair_create(RandomWalk(2,t),w))

24

In [12]:
idx_pairs=[]
for i in range(len(v)):
    for j in range(y):
        idx_pairs.append(pair_create(RandomWalk(i,t),w))

In [14]:
idx_pairs=np.array([np.array(i) for i in idx_pairs])
idx_pairs=idx_pairs.reshape(-1,2)
idx_pairs.shape

(336, 2)

In [15]:
def get_input_layer(node_idx):
    x = torch.zeros(len(v)).float()
    x[node_idx] = 1.0
    return x

In [16]:
embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, len(v)).float(), requires_grad=True)
W2 = Variable(torch.randn(len(v), embedding_dims).float(), requires_grad=True)
num_epochs = 100
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data[0]
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

/home/shubham/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Loss at epo 0: 3.1456782817840576
Loss at epo 10: 2.183809280395508
Loss at epo 20: 1.9378072023391724
Loss at epo 30: 1.831724762916565
Loss at epo 40: 1.7757853269577026
Loss at epo 50: 1.7429765462875366
Loss at epo 60: 1.7220067977905273
Loss at epo 70: 1.7074934244155884
Loss at epo 80: 1.6967012882232666
Loss at epo 90: 1.688170313835144


In [18]:
W1,W2

(tensor([[-0.4268, -0.7080, -0.2460, -0.0298,  0.5165,  0.0486, -0.2550],
         [ 0.4321, -1.6612, -1.1200,  0.4259, -0.2870,  0.7627, -0.6278],
         [ 0.8007,  0.4945,  0.5461,  0.0590, -0.2820,  0.8470, -0.0704],
         [-0.4764,  0.0360, -1.1690,  0.8348, -0.3476,  0.6047,  0.4120],
         [ 0.1563,  1.0724,  0.9587,  0.3117,  0.3879,  1.6632, -0.8993]]),
 tensor([[-0.2597, -1.4178,  0.0346,  0.1491, -0.9558],
         [-1.1522, -0.6324,  0.8297, -0.6346, -0.9648],
         [-1.0262, -0.9026, -0.3238,  0.3878, -0.0274],
         [ 0.2300, -0.5019, -1.7414, -0.5677,  0.8247],
         [ 1.2475, -0.5316,  0.2394,  0.9477, -0.2287],
         [ 2.1535, -0.7676,  0.1213,  0.7389, -0.7456],
         [-2.5345,  0.2554, -0.5300, -0.2536, -0.1836]]))